In [ ]:
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier


# Additional library
import os
import re
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
!pip install google-trans-new
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=099b7070b65b56cf668baa2803fb7f068234d3e49a1e72f44026b4850cf34f1c
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling char

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_1 = pd.read_excel('/content/drive/My Drive/Skripsi/Data Crawling Syiah.xlsx')
df_2 = pd.read_excel('/content/drive/My Drive/Skripsi/Data Crawling Syiah Indonesia.xlsx')
df_3 = pd.read_excel('/content/drive/My Drive/Skripsi/Data Crawling Kafir.xlsx')
df_4 = pd.read_excel('/content/drive/My Drive/Skripsi/Data Crawling Sesat.xlsx')
df_5 = pd.read_excel('/content/drive/My Drive/Skripsi/Data Crawling Diluar Islam.xlsx')
df_6 = pd.read_excel('/content/drive/My Drive/Skripsi/Data Crawling Islam.xlsx')
df_7 = pd.read_excel('/content/drive/My Drive/Skripsi/Data Crawling Masih Islam.xlsx')

In [ ]:
#merge dataset
frames = [df_1, df_2, df_3, df_4, df_5, df_6, df_7]
df = pd.concat(frames)

#replace column's name
df = df.rename(columns={"pengguna" :"pengguna", "tweet": "tweet"})
df = df[['pengguna','tweet']]
df.head()

,pengguna,tweet
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ..."


Data Cleansing

In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

data cleansing process including Removing Punctuation inside

In [ ]:
# Menghilangkan karakter khusus dan angka / REMOVING PUNCTUATION
def cleansing(text):
    text = ' '.join(re.sub("[@#][A-Za-z0-9_]+|([^0-9A-Za-z \t])|(\w+:\/\/\S+)+|^https?:\/\/.*[\r\n]*"," ", text).split(' '))
    text = re.sub(r"\d+", "", text)
    text = ' '.join(text.split())
    return text

In [ ]:
list_tweet = list()
for index, row in df.iterrows():
    _tweet = row['tweet']
    # _tweet = caseFolding(_tweet)
    _tweet = cleansing(_tweet)

    tmp_list = {
        #'id'          : row['id'],
        'pengguna'    : row['pengguna'],
        'tweet'       : row['tweet'],
        'clean_tweet' : _tweet
    }
    list_tweet.append(tmp_list)

df_list_tweet = pd.DataFrame(list_tweet)
df_list_tweet['clean_tweet']=df_list_tweet['clean_tweet'].replace(regex={'RT': ''})

In [ ]:
df_list_tweet

,pengguna,tweet,clean_tweet
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...
...,...,...,...
255,akukash,RT @abuhafiz1: Masuk kedalam Kaabah adalah pel...,Masuk kedalam Kaabah adalah peluang nikmat ya...
256,hajardottcom,RT @abuhafiz1: Masuk kedalam Kaabah adalah pel...,Masuk kedalam Kaabah adalah peluang nikmat ya...
257,azrylzul16,RT @abuhafiz1: Masuk kedalam Kaabah adalah pel...,Masuk kedalam Kaabah adalah peluang nikmat ya...
258,zvvko,RT @abuhafiz1: Masuk kedalam Kaabah adalah pel...,Masuk kedalam Kaabah adalah peluang nikmat ya...


In [ ]:
# Menghilangkan tweet yang sama dari data hasil cleansing
new_list_tweet = df_list_tweet.drop_duplicates(subset=['clean_tweet'])
print('Total tweet sebelum sebelum duplicate :', len(df_list_tweet))
print('Total tweet sesudah sesudah remove duplicate :', len(new_list_tweet))

df_new_list_tweet = pd.DataFrame(new_list_tweet)
df_new_list_tweet

Total tweet sebelum sebelum duplicate : 260
Total tweet sesudah sesudah remove duplicate : 94


,pengguna,tweet,clean_tweet
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...
...,...,...,...
231,wahyu_djanti88,@JanissaryD_Last @islah_bahrawi @tni_ad Kaum l...,Kaum liberal syiah atheis setengah mateng teru...
232,theonlyoneLhMY,RT @KulupFans: Pihak jabatan agama islam Johor...,Pihak jabatan agama islam Johor dah boleh kut...
239,infoahlulbait,Doa Hari Ke-6 Ramadhan\n\n#Islam #Doa #Ramadha...,Doa Hari Ke Ramadhan
240,boothefooo,@satrio_em @James_Al_Murtad @agniasambara Tanp...,Tanpa bermaksud belain mba nya ataupun belain ...


In [ ]:
#export df_new_list_tweet to google drive
df_new_list_tweet.to_excel('/content/drive/My Drive/Skripsi/df_new_list_tweet_2.1.xlsx')

In [ ]:
!pip install googletrans
!pip install google-trans-new
!pip install google_trans_new

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
# Contoh eng to ind
from googletrans import Translator
translator = Translator()
translator.translate('This sentence is written in English', dest='id').text

'Kalimat ini ditulis dalam bahasa Inggris'

In [ ]:
# Contoh ind to eng
from googletrans import Translator
translator = Translator()
translator.translate('Kalimat ini ditulis dalam bahasa Inggris', dest='en').text

'This sentence is written in English'

In [ ]:
translated_data = df_new_list_tweet
translated_data.head()

,pengguna,tweet,clean_tweet
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...


In [ ]:
#translate tweets to English
from googletrans import Translator
translator = Translator()

translations = []

for i in translated_data.index:
  translated = translator.translate(translated_data.clean_tweet[i], dest='en').text
  translations.append(translated)

In [ ]:
# tampung hasil translate ke variable/dataframe translations
translations = pd.DataFrame(translations)

# gabungkan hasli translate kedalam data translated_data
translated_data['en_tweet'] = translations

In [ ]:
translated_data

,pengguna,tweet,clean_tweet,en_tweet
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...,Do you have sympathy for the children in North...
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...,I don't think it's a Muslim but a Shia so don'...
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...,"It's been a year since the invasion of Yemen, ..."
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...,Hopefully the last time before being suspended...
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...,"Wow, even though the Saudis are only backing u..."
...,...,...,...,...
231,wahyu_djanti88,@JanissaryD_Last @islah_bahrawi @tni_ad Kaum l...,Kaum liberal syiah atheis setengah mateng teru...,NaN
232,theonlyoneLhMY,RT @KulupFans: Pihak jabatan agama islam Johor...,Pihak jabatan agama islam Johor dah boleh kut...,NaN
239,infoahlulbait,Doa Hari Ke-6 Ramadhan\n\n#Islam #Doa #Ramadha...,Doa Hari Ke Ramadhan,NaN
240,boothefooo,@satrio_em @James_Al_Murtad @agniasambara Tanp...,Tanpa bermaksud belain mba nya ataupun belain ...,NaN


In [ ]:
#export df_new_list_tweet to google drive
translated_data.to_excel('/content/drive/My Drive/Skripsi/translated_data_tweets_2.1.xlsx', index=False)

Data Pre-processing

In [ ]:
# Import Library
import pandas as pd
import numpy as np

In [ ]:
!pip install nltk

In [ ]:
# word_tokenize library for tokenization process using nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

# library for process of pos-tagging, lemmatization and cleaning the stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer # pembobotan kemudian u/ menghitung polarity
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
sample_reviews = pd.read_excel('/content/drive/My Drive/Skripsi/translated_data_tweets_2.1.xlsx')

In [ ]:
df_corpus = sample_reviews
df_corpus.head()

,pengguna,tweet,clean_tweet,en_tweet
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...,Do you have sympathy for the children in North...
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...,I don't think it's a Muslim but a Shia so don'...
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...,"It's been a year since the invasion of Yemen, ..."
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...,Hopefully the last time before being suspended...
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...,"Wow, even though the Saudis are only backing u..."


CASE FOLDING

In [ ]:
#konversi tweet ke LOWER CASE / CASE FOLDING
df_corpus['text'] = df_corpus['en_tweet'].str.lower()

In [ ]:
# cek data null
# word_tokenize tidak akan jalan kalau data masih ada yang null
# data tweet null karena ada user yang tidak mention apapun

df_corpus.isnull().sum()

pengguna        0
tweet           0
clean_tweet     0
en_tweet       35
text           35
dtype: int64

In [ ]:
df_corpus.head(5)

,pengguna,tweet,clean_tweet,en_tweet,text
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...,Do you have sympathy for the children in North...,do you have sympathy for the children in north...
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...,I don't think it's a Muslim but a Shia so don'...,i don't think it's a muslim but a shia so don'...
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...,"It's been a year since the invasion of Yemen, ...","it's been a year since the invasion of yemen, ..."
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...,Hopefully the last time before being suspended...,hopefully the last time before being suspended...
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...,"Wow, even though the Saudis are only backing u...","wow, even though the saudis are only backing u..."


TOKENIZATION

In [ ]:
#drop data null
corpus = df_corpus.dropna()

#konversi tweet ke proses TOKENIZATION menggunakan word_tokenize
corpus['text_tokenized'] = corpus.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)

<ipython-input-35-3a15ddd33eb1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus['text_tokenized'] = corpus.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)


In [ ]:
corpus.head()

,pengguna,tweet,clean_tweet,en_tweet,text,text_tokenized
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...,Do you have sympathy for the children in North...,do you have sympathy for the children in north...,"[do, you, have, sympathy, for, the, children, ..."
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...,I don't think it's a Muslim but a Shia so don'...,i don't think it's a muslim but a shia so don'...,"[i, do, n't, think, it, 's, a, muslim, but, a,..."
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...,"It's been a year since the invasion of Yemen, ...","it's been a year since the invasion of yemen, ...","[it, 's, been, a, year, since, the, invasion, ..."
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...,Hopefully the last time before being suspended...,hopefully the last time before being suspended...,"[hopefully, the, last, time, before, being, su..."
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...,"Wow, even though the Saudis are only backing u...","wow, even though the saudis are only backing u...","[wow, ,, even, though, the, saudis, are, only,..."


In [ ]:
corpus.isnull().sum()

pengguna          0
tweet             0
clean_tweet       0
en_tweet          0
text              0
text_tokenized    0
dtype: int64

POS-TAGGING

In [ ]:
#create function for pos-tagging
import nltk

def tokenization(text):
  sentence = nltk.sent_tokenize(text)
  for sent in sentence:
    pos_tagging = nltk.pos_tag(nltk.word_tokenize(sent))
  return pos_tagging

array_baru = []
looping = corpus['text']
for text in looping:
  array_baru.append(tokenization(text))
# print(array_baru)

#apply function for pos-tagging
corpus['text_tagging']  = array_baru

<ipython-input-38-7a8e772fdac4>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus['text_tagging']  = array_baru


In [ ]:
corpus.head()

,pengguna,tweet,clean_tweet,en_tweet,text,text_tokenized,text_tagging
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...,Do you have sympathy for the children in North...,do you have sympathy for the children in north...,"[do, you, have, sympathy, for, the, children, ...","[(do, VBP), (you, PRP), (have, VB), (sympathy,..."
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...,I don't think it's a Muslim but a Shia so don'...,i don't think it's a muslim but a shia so don'...,"[i, do, n't, think, it, 's, a, muslim, but, a,...","[(i, NNS), (do, VBP), (n't, RB), (think, VB), ..."
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...,"It's been a year since the invasion of Yemen, ...","it's been a year since the invasion of yemen, ...","[it, 's, been, a, year, since, the, invasion, ...","[(it, PRP), ('s, VBZ), (been, VBN), (a, DT), (..."
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...,Hopefully the last time before being suspended...,hopefully the last time before being suspended...,"[hopefully, the, last, time, before, being, su...","[(hopefully, RB), (the, DT), (last, JJ), (time..."
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...,"Wow, even though the Saudis are only backing u...","wow, even though the saudis are only backing u...","[wow, ,, even, though, the, saudis, are, only,...","[(wow, NN), (,, ,), (even, RB), (though, IN), ..."


STOPWORD

In [ ]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [ ]:
#create function removing stopword
def remove_stopwords(txt_tokenized):
  txt_clean = [word for word in txt_tokenized if word not in stopwords]
  return txt_clean

#apply function for removing stopword
corpus['text_stopword'] = corpus['text_tokenized'].apply(lambda x: remove_stopwords(x))

<ipython-input-41-a180ec132bad>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus['text_stopword'] = corpus['text_tokenized'].apply(lambda x: remove_stopwords(x))


In [ ]:
corpus.head()

,pengguna,tweet,clean_tweet,en_tweet,text,text_tokenized,text_tagging,text_stopword
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...,Do you have sympathy for the children in North...,do you have sympathy for the children in north...,"[do, you, have, sympathy, for, the, children, ...","[(do, VBP), (you, PRP), (have, VB), (sympathy,...","[sympathy, children, northern, pakistan, whose..."
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...,I don't think it's a Muslim but a Shia so don'...,i don't think it's a muslim but a shia so don'...,"[i, do, n't, think, it, 's, a, muslim, but, a,...","[(i, NNS), (do, VBP), (n't, RB), (think, VB), ...","[n't, think, 's, muslim, shia, n't, surprised,..."
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...,"It's been a year since the invasion of Yemen, ...","it's been a year since the invasion of yemen, ...","[it, 's, been, a, year, since, the, invasion, ...","[(it, PRP), ('s, VBZ), (been, VBN), (a, DT), (...","['s, year, since, invasion, yemen, ,, sunnah, ..."
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...,Hopefully the last time before being suspended...,hopefully the last time before being suspended...,"[hopefully, the, last, time, before, being, su...","[(hopefully, RB), (the, DT), (last, JJ), (time...","[hopefully, last, time, suspended, ,, critical..."
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...,"Wow, even though the Saudis are only backing u...","wow, even though the saudis are only backing u...","[wow, ,, even, though, the, saudis, are, only,...","[(wow, NN), (,, ,), (even, RB), (though, IN), ...","[wow, ,, even, though, saudis, backing, yemeni..."


STEMMING
Post Stemmer

In [ ]:
import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()
dir(ps) # use stem function bellow

['MARTIN_EXTENSIONS',
 'NLTK_EXTENSIONS',
 'ORIGINAL_ALGORITHM',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_apply_rule_list',
 '_contains_vowel',
 '_ends_cvc',
 '_ends_double_consonant',
 '_has_positive_measure',
 '_is_consonant',
 '_measure',
 '_replace_suffix',
 '_step1a',
 '_step1b',
 '_step1c',
 '_step2',
 '_step3',
 '_step4',
 '_step5a',
 '_step5b',
 'mode',
 'pool',
 'stem',
 'vowels']

In [ ]:
#exaxmple steaming function
print(ps.stem('coder'))
print(ps.stem('codeing'))
print(ps.stem('code'))

coder
code
code


In [ ]:
#create function for stemming
def stemming(tokenized_text):
  text = [ps.stem(word) for word in tokenized_text]
  return text

#apply function for removing stopword
corpus['text_stemmed'] = corpus['text_stopword'].apply(lambda x: stemming(x))

<ipython-input-45-cd203cc59862>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus['text_stemmed'] = corpus['text_stopword'].apply(lambda x: stemming(x))


In [ ]:
corpus.head()

,pengguna,tweet,clean_tweet,en_tweet,text,text_tokenized,text_tagging,text_stopword,text_stemmed
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...,Do you have sympathy for the children in North...,do you have sympathy for the children in north...,"[do, you, have, sympathy, for, the, children, ...","[(do, VBP), (you, PRP), (have, VB), (sympathy,...","[sympathy, children, northern, pakistan, whose...","[sympathi, children, northern, pakistan, whose..."
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...,I don't think it's a Muslim but a Shia so don'...,i don't think it's a muslim but a shia so don'...,"[i, do, n't, think, it, 's, a, muslim, but, a,...","[(i, NNS), (do, VBP), (n't, RB), (think, VB), ...","[n't, think, 's, muslim, shia, n't, surprised,...","[n't, think, 's, muslim, shia, n't, surpris, s..."
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...,"It's been a year since the invasion of Yemen, ...","it's been a year since the invasion of yemen, ...","[it, 's, been, a, year, since, the, invasion, ...","[(it, PRP), ('s, VBZ), (been, VBN), (a, DT), (...","['s, year, since, invasion, yemen, ,, sunnah, ...","['s, year, sinc, invas, yemen, ,, sunnah, shia..."
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...,Hopefully the last time before being suspended...,hopefully the last time before being suspended...,"[hopefully, the, last, time, before, being, su...","[(hopefully, RB), (the, DT), (last, JJ), (time...","[hopefully, last, time, suspended, ,, critical...","[hope, last, time, suspend, ,, critic, spirit,..."
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...,"Wow, even though the Saudis are only backing u...","wow, even though the saudis are only backing u...","[wow, ,, even, though, the, saudis, are, only,...","[(wow, NN), (,, ,), (even, RB), (though, IN), ...","[wow, ,, even, though, saudis, backing, yemeni...","[wow, ,, even, though, saudi, back, yemeni, go..."


In [ ]:
#Export
corpus.to_excel('/content/drive/My Drive/Skripsi/corpus_without_lemmatization_2.1.xlsx',index=False)

Labeling dengan TextBlob

In [ ]:
import pandas as pd
import numpy as np
import csv
import re
from datetime import datetime

# menghitung sentimen kata-kata yang berbahasa inggris
from textblob import TextBlob

In [ ]:
corpus_2 = pd.read_excel('/content/drive/My Drive/Skripsi/corpus_without_lemmatization_2.1.xlsx')
corpus_2.head()

,pengguna,tweet,clean_tweet,en_tweet,text,text_tokenized,text_tagging,text_stopword,text_stemmed
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...,Do you have sympathy for the children in North...,do you have sympathy for the children in north...,"['do', 'you', 'have', 'sympathy', 'for', 'the'...","[('do', 'VBP'), ('you', 'PRP'), ('have', 'VB')...","['sympathy', 'children', 'northern', 'pakistan...","['sympathi', 'children', 'northern', 'pakistan..."
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...,I don't think it's a Muslim but a Shia so don'...,i don't think it's a muslim but a shia so don'...,"['i', 'do', ""n't"", 'think', 'it', ""'s"", 'a', '...","[('i', 'NNS'), ('do', 'VBP'), (""n't"", 'RB'), (...","[""n't"", 'think', ""'s"", 'muslim', 'shia', ""n't""...","[""n't"", 'think', ""'s"", 'muslim', 'shia', ""n't""..."
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...,"It's been a year since the invasion of Yemen, ...","it's been a year since the invasion of yemen, ...","['it', ""'s"", 'been', 'a', 'year', 'since', 'th...","[('it', 'PRP'), (""'s"", 'VBZ'), ('been', 'VBN')...","[""'s"", 'year', 'since', 'invasion', 'yemen', '...","[""'s"", 'year', 'sinc', 'invas', 'yemen', ',', ..."
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...,Hopefully the last time before being suspended...,hopefully the last time before being suspended...,"['hopefully', 'the', 'last', 'time', 'before',...","[('hopefully', 'RB'), ('the', 'DT'), ('last', ...","['hopefully', 'last', 'time', 'suspended', ','...","['hope', 'last', 'time', 'suspend', ',', 'crit..."
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...,"Wow, even though the Saudis are only backing u...","wow, even though the saudis are only backing u...","['wow', ',', 'even', 'though', 'the', 'saudis'...","[('wow', 'NN'), (',', ','), ('even', 'RB'), ('...","['wow', ',', 'even', 'though', 'saudis', 'back...","['wow', ',', 'even', 'though', 'saudi', 'back'..."


In [ ]:
#determine the polarity value from dataset
sample_reviews = corpus_2

def detect_polarity(text_stemmed):
    return TextBlob(text_stemmed).sentiment.polarity

sample_reviews['polarity'] = sample_reviews.text_stemmed.apply(detect_polarity)
sample_reviews.head()

,pengguna,tweet,clean_tweet,en_tweet,text,text_tokenized,text_tagging,text_stopword,text_stemmed,polarity
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...,Do you have sympathy for the children in North...,do you have sympathy for the children in north...,"['do', 'you', 'have', 'sympathy', 'for', 'the'...","[('do', 'VBP'), ('you', 'PRP'), ('have', 'VB')...","['sympathy', 'children', 'northern', 'pakistan...","['sympathi', 'children', 'northern', 'pakistan...",-0.05
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...,I don't think it's a Muslim but a Shia so don'...,i don't think it's a muslim but a shia so don'...,"['i', 'do', ""n't"", 'think', 'it', ""'s"", 'a', '...","[('i', 'NNS'), ('do', 'VBP'), (""n't"", 'RB'), (...","[""n't"", 'think', ""'s"", 'muslim', 'shia', ""n't""...","[""n't"", 'think', ""'s"", 'muslim', 'shia', ""n't""...",0.00
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...,"It's been a year since the invasion of Yemen, ...","it's been a year since the invasion of yemen, ...","['it', ""'s"", 'been', 'a', 'year', 'since', 'th...","[('it', 'PRP'), (""'s"", 'VBZ'), ('been', 'VBN')...","[""'s"", 'year', 'since', 'invasion', 'yemen', '...","[""'s"", 'year', 'sinc', 'invas', 'yemen', ',', ...",0.00
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...,Hopefully the last time before being suspended...,hopefully the last time before being suspended...,"['hopefully', 'the', 'last', 'time', 'before',...","[('hopefully', 'RB'), ('the', 'DT'), ('last', ...","['hopefully', 'last', 'time', 'suspended', ','...","['hope', 'last', 'time', 'suspend', ',', 'crit...",0.00
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...,"Wow, even though the Saudis are only backing u...","wow, even though the saudis are only backing u...","['wow', ',', 'even', 'though', 'the', 'saudis'...","[('wow', 'NN'), (',', ','), ('even', 'RB'), ('...","['wow', ',', 'even', 'though', 'saudis', 'back...","['wow', ',', 'even', 'though', 'saudi', 'back'...",0.05






*   Seperti yang bisa dilihat pada source code diatas, attribute text_stemmed yang akan dipilih untuk menentukan polarity.
*   Statement row dari attribute text_stemmed akan menerima nilai index untuk statment polarity secara otomatis.
*   Nilai polarity akan membantu dalam proses penentuan kelas label.





In [ ]:
#Membuat functionnya untuk memetakan label sentiment dari nilai polarity
def convert_label_category(row):
  if row['polarity'] > 0:
    return 'positif'
  elif row['polarity'] == 0:
    return 'netral'
  else:
    return 'negatif'

#Apply the function into dataframe
sample_reviews['label'] = sample_reviews.apply(convert_label_category,axis=1)

In [ ]:
sample_reviews

,pengguna,tweet,clean_tweet,en_tweet,text,text_tokenized,text_tagging,text_stopword,text_stemmed,polarity,label
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...,Do you have sympathy for the children in North...,do you have sympathy for the children in north...,"['do', 'you', 'have', 'sympathy', 'for', 'the'...","[('do', 'VBP'), ('you', 'PRP'), ('have', 'VB')...","['sympathy', 'children', 'northern', 'pakistan...","['sympathi', 'children', 'northern', 'pakistan...",-0.050000,negatif
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...,I don't think it's a Muslim but a Shia so don'...,i don't think it's a muslim but a shia so don'...,"['i', 'do', ""n't"", 'think', 'it', ""'s"", 'a', '...","[('i', 'NNS'), ('do', 'VBP'), (""n't"", 'RB'), (...","[""n't"", 'think', ""'s"", 'muslim', 'shia', ""n't""...","[""n't"", 'think', ""'s"", 'muslim', 'shia', ""n't""...",0.000000,netral
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...,"It's been a year since the invasion of Yemen, ...","it's been a year since the invasion of yemen, ...","['it', ""'s"", 'been', 'a', 'year', 'since', 'th...","[('it', 'PRP'), (""'s"", 'VBZ'), ('been', 'VBN')...","[""'s"", 'year', 'since', 'invasion', 'yemen', '...","[""'s"", 'year', 'sinc', 'invas', 'yemen', ',', ...",0.000000,netral
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...,Hopefully the last time before being suspended...,hopefully the last time before being suspended...,"['hopefully', 'the', 'last', 'time', 'before',...","[('hopefully', 'RB'), ('the', 'DT'), ('last', ...","['hopefully', 'last', 'time', 'suspended', ','...","['hope', 'last', 'time', 'suspend', ',', 'crit...",0.000000,netral
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...,"Wow, even though the Saudis are only backing u...","wow, even though the saudis are only backing u...","['wow', ',', 'even', 'though', 'the', 'saudis'...","[('wow', 'NN'), (',', ','), ('even', 'RB'), ('...","['wow', ',', 'even', 'though', 'saudis', 'back...","['wow', ',', 'even', 'though', 'saudi', 'back'...",0.050000,positif
5,boothefooo,Ribetnya itu kan soalnya Quranist selalu ditod...,Ribetnya itu kan soalnya Quranist selalu ditod...,It's complicated because Koranists are always ...,it's complicated because koranists are always ...,"['it', ""'s"", 'complicated', 'because', 'korani...","[('oh', 'UH'), (',', ','), ('if', 'IN'), ('you...","[""'s"", 'complicated', 'koranists', 'always', '...","[""'s"", 'complic', 'koranist', 'alway', 'held',...",-0.312500,negatif
6,youdie_HIMMLER,RT @CeritaPuspa: Aku tuch orangnya gak pelit f...,Aku tuch orangnya gak pelit follow amp gak pe...,I'm a person who is not stingy with following ...,i'm a person who is not stingy with following ...,"['i', ""'m"", 'a', 'person', 'who', 'is', 'not',...","[('i', 'NN'), (""'m"", 'VBP'), ('a', 'DT'), ('pe...","[""'m"", 'person', 'stingy', 'following', '&', '...","[""'m"", 'person', 'stingi', 'follow', '&', 'sti...",0.000000,netral
7,roniandriska,RT @jandakembang78: @YandraD31 Ini tentara Syi...,Ini tentara Syiah nya Bashar al Assad Tp kela...,"This is Bashar al Assad's Shiite army, but Isr...","this is bashar al assad's shiite army, but isr...","['this', 'is', 'bashar', 'al', 'assad', ""'s"", ...","[('this', 'DT'), ('is', 'VBZ'), ('bashar', 'JJ...","['bashar', 'al', 'assad', ""'s"", 'shiite', 'arm...","['bashar', 'al', 'assad', ""'s"", 'shiit', 'armi...",-0.700000,negatif
8,Bengkel_MP,"RT @infoahlulbait: Bulan Ramadhan, Kesempatan ...",Bulan Ramadhan Kesempatan Terbaik Berdamai de...,The month of Ramadhan is the best opportunity ...,the month of ramadhan is the best opportunity ...,"['the', 'month', 'of', 'ramadhan', 'is', 'the'...","[('allah', 'NN'), ('swt', 'NN'), ('in', 'IN'),...","['m

In [ ]:
#Hasil dari penentuan kelas label dengan TextBlob
sample_reviews['label'].value_counts()

netral     31
positif    17
negatif    11
Name: label, dtype: int64

In [ ]:
sample_reviews[sample_reviews['label']=='negatif'].head(3)

,pengguna,tweet,clean_tweet,en_tweet,text,text_tokenized,text_tagging,text_stopword,text_stemmed,polarity,label
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...,Do you have sympathy for the children in North...,do you have sympathy for the children in north...,"['do', 'you', 'have', 'sympathy', 'for', 'the'...","[('do', 'VBP'), ('you', 'PRP'), ('have', 'VB')...","['sympathy', 'children', 'northern', 'pakistan...","['sympathi', 'children', 'northern', 'pakistan...",-0.0500,negatif
5,boothefooo,Ribetnya itu kan soalnya Quranist selalu ditod...,Ribetnya itu kan soalnya Quranist selalu ditod...,It's complicated because Koranists are always ...,it's complicated because koranists are always ...,"['it', ""'s"", 'complicated', 'because', 'korani...","[('oh', 'UH'), (',', ','), ('if', 'IN'), ('you...","[""'s"", 'complicated', 'koranists', 'always', '...","[""'s"", 'complic', 'koranist', 'alway', 'held',...",-0.3125,negatif
7,roniandriska,RT @jandakembang78: @YandraD31 Ini tentara Syi...,Ini tentara Syiah nya Bashar al Assad Tp kela...,"This is Bashar al Assad's Shiite army, but Isr...","this is bashar al assad's shiite army, but isr...","['this', 'is', 'bashar', 'al', 'assad', ""'s"", ...","[('this', 'DT'), ('is', 'VBZ'), ('bashar', 'JJ...","['bashar', 'al', 'assad', ""'s"", 'shiite', 'arm...","['bashar', 'al', 'assad', ""'s"", 'shiit', 'armi...",-0.7000,negatif


In [ ]:
#Export analisis sentiment dan pnentuan kelas label menggunakan text blob
sample_reviews.to_excel('/content/drive/My Drive/Skripsi/sample_reviews_2.1_nolemma.xlsx',index=False)

In [ ]:
df = pd.read_excel('/content/drive/My Drive/Skripsi/sample_reviews_2.1_nolemma.xlsx')
dfclean = df.copy()
dfclean.head(2)

,pengguna,tweet,clean_tweet,en_tweet,text,text_tokenized,text_tagging,text_stopword,text_stemmed,polarity,label
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...,Do you have sympathy for the children in North...,do you have sympathy for the children in north...,"['do', 'you', 'have', 'sympathy', 'for', 'the'...","[('do', 'VBP'), ('you', 'PRP'), ('have', 'VB')...","['sympathy', 'children', 'northern', 'pakistan...","['sympathi', 'children', 'northern', 'pakistan...",-0.05,negatif
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...,I don't think it's a Muslim but a Shia so don'...,i don't think it's a muslim but a shia so don'...,"['i', 'do', ""n't"", 'think', 'it', ""'s"", 'a', '...","[('i', 'NNS'), ('do', 'VBP'), (""n't"", 'RB'), (...","[""n't"", 'think', ""'s"", 'muslim', 'shia', ""n't""...","[""n't"", 'think', ""'s"", 'muslim', 'shia', ""n't""...",0.00,netral


In [ ]:
dfclean.head()

,pengguna,tweet,clean_tweet,en_tweet,text,text_tokenized,text_tagging,text_stopword,text_stemmed,polarity,label
0,towongsing,RT @pengarang_sajak: Adakah simpati utk anak2 ...,Adakah simpati utk anak di Utara Pakistan yg ...,Do you have sympathy for the children in North...,do you have sympathy for the children in north...,"['do', 'you', 'have', 'sympathy', 'for', 'the'...","[('do', 'VBP'), ('you', 'PRP'), ('have', 'VB')...","['sympathy', 'children', 'northern', 'pakistan...","['sympathi', 'children', 'northern', 'pakistan...",-0.05,negatif
1,fhyetan,@Nebuchanedzar_I @mudasir_11 @UpdateBolabola G...,Gua kira itu bukan orang islam tapi Syiah jadi...,I don't think it's a Muslim but a Shia so don'...,i don't think it's a muslim but a shia so don'...,"['i', 'do', ""n't"", 'think', 'it', ""'s"", 'a', '...","[('i', 'NNS'), ('do', 'VBP'), (""n't"", 'RB'), (...","[""n't"", 'think', ""'s"", 'muslim', 'shia', ""n't""...","[""n't"", 'think', ""'s"", 'muslim', 'shia', ""n't""...",0.00,netral
2,MHHBJA2005,Genap 8 tahun buat serangan pencerobohan ke at...,Genap tahun buat serangan pencerobohan ke atas...,"It's been a year since the invasion of Yemen, ...","it's been a year since the invasion of yemen, ...","['it', ""'s"", 'been', 'a', 'year', 'since', 'th...","[('it', 'PRP'), (""'s"", 'VBZ'), ('been', 'VBN')...","[""'s"", 'year', 'since', 'invasion', 'yemen', '...","[""'s"", 'year', 'sinc', 'invas', 'yemen', ',', ...",0.00,netral
3,HarryRusuh2,@NdaJoesh @HarryRidwan__Ay Mudah mudahan. Tera...,Mudah mudahan Terakhir sebelom suspend Tadi ny...,Hopefully the last time before being suspended...,hopefully the last time before being suspended...,"['hopefully', 'the', 'last', 'time', 'before',...","[('hopefully', 'RB'), ('the', 'DT'), ('last', ...","['hopefully', 'last', 'time', 'suspended', ','...","['hope', 'last', 'time', 'suspend', ',', 'crit...",0.00,netral
4,ahmaddiontri,"@andy24670125 @fadlizon Wkwkwk, padahal Saudi ...",Wkwkwk padahal Saudi hanya back up pemerintaha...,"Wow, even though the Saudis are only backing u...","wow, even though the saudis are only backing u...","['wow', ',', 'even', 'though', 'the', 'saudis'...","[('wow', 'NN'), (',', ','), ('even', 'RB'), ('...","['wow', ',', 'even', 'though', 'saudis', 'back...","['wow', ',', 'even', 'though', 'saudi', 'back'...",0.05,positif
